# Using a trained reflectorch model

## Loading the trained model

The first step is importing the necessary methods from the reflectorch package, as well as othar basic Python packages:

In [1]:
import torch
import matplotlib.pyplot as plt

from ipywidgets import interact
from reflectorch import get_trainer_by_name

torch.manual_seed(0); # set seed for reproducibility

In order to import a trained reflectorch model, we first have to specify the name of the trained model we wish to load. This name should match the name of the YAML configuration file used for training that model. Here we load a model for a 2-layer box parameterization of the thin film SLD profile.

In [2]:
trained_model_name = 'c1_trained'

Next, we initialize an instance of the `PointEstimatorTrainer` class using the `get_trainer_by_name` method.

:::{warning}
The `load_weights` argument must be set to `True` in order for the saved weights of the neural network to be loaded, otherwise the network weights are randomly initialized.
:::

In [3]:
trainer = get_trainer_by_name(config_name=trained_model_name, load_weights=True)

Model c1_trained loaded. Number of parameters: 3.83 M


## Generating simulated data

We can generate a batch of simulated data using the `get_batch` method of the data loader:

In [4]:
batch_size = 64
simulated_data = trainer.loader.get_batch(batch_size=batch_size)

This method returns a dictionary with 4 entries indexed by the following keys:

 1. **params** - an instance of the `ParametricParams` class containing the original (unscaled) values of the generated parameters, the generated minimum prior bound for each parameter and the generated maximum prior bound for each parameter (see the [paper](https://doi.org/10.1107/S1600576724002115) for more details about the generation process)
 2. **scaled_params** - a Pytorch Tensor containing the parameters, minimum bounds and maximum bounds, all scaled to the ML-friendly range [-1, 1]
 3. **q_values** - a Pytorch Tensor containing the reciprocal space (q) positions of the points in the reflectivity curve, in units of Å<sup>-1</sup>
 4. **scaled_noisy_curves** - a Pytorch Tensor containing the simulated reflectivity curves (including added noise) scaled to the ML-friendly range [-1, 1]

We can inspect some of the (scaled) simulated curves: 

(*interactive cursor not supported in current version of Jupyter Book*)

In [5]:
q = simulated_data['q_values']
scaled_noisy_curves = simulated_data['scaled_noisy_curves']
unscaled_noisy_curves = trainer.loader.curves_scaler.restore(scaled_noisy_curves)

@interact(i=(0, batch_size-1, 1))
def plot_refl_curve(i=0):
    
    fig, ax = plt.subplots(1,1,figsize=(6,6))
    ax.set_ylim(-1.1, 1.1)
    ax.set_xlabel('q [$Å^{-1}$]', fontsize=24)
    ax.set_ylabel('R$_{scaled}$ (q)', fontsize=24)
    ax.tick_params(axis='both', which='major', labelsize=15)
        
    ax.scatter(q[i].cpu().numpy(), scaled_noisy_curves[i].cpu().numpy(), c='blue', s=2.0)

interactive(children=(IntSlider(value=0, description='i', max=63), Output()), _dom_classes=('widget-interact',…

This trained model corresponds to a 2 layer (*in addition to the substrate*) parameterization of the SLD profile, which corresponds to 8 predicted film parameters:

In [6]:
n_layers = simulated_data['params'].max_layer_num
n_params = simulated_data['params'].num_params

print(f'Number of layers: {n_layers},  Number of film parameters: {n_params}')

Number of layers: 2,  Number of film parameters: 8


## Applying the model to simulated data

The input to the neural network consists in the batch of reflectivity curves together with the prior bounds (minimum and maximum) for each film parameter. For experimental data, the prior bounds can be set according to the prior knowledge about the investigated thin film. In this example on simulated data, we use the prior bounds already sampled during the data generation process (i.e. meant for training the model) which ensures reasonable values for the prior bounds. 

In the `scaled_params` tensor the first 8 columns correspond to the *scaled* ground truth values of the film parameters, the next 8 columns to the *scaled* minimum bounds for the parameters and the last 8 to the *scaled* maximum bounds for the parameters. Thus, we select the last 16 columns as our input prior bounds:

In [7]:
scaled_bounds = simulated_data['scaled_params'][..., n_params:]

print(scaled_bounds.shape)

torch.Size([64, 16])


To obtain the input to the neural network, we concatenate the the reflectiviy curves with the prior bounds along the last axis of the tensor (`dim=-1`). We should also make sure that the input is of the `float` data type.

In [8]:
scaled_input = torch.cat([scaled_noisy_curves, scaled_bounds], dim=-1).float()

print(scaled_input.shape)

torch.Size([64, 144])


The neural network can be accessed as the `model` attribute of the trainer. By providing the previously constructed scaled input (reflectivity curves + prior bounds) to the network, we obtain the predictions for the parameters, scaled with respect to the prior bounds.

:::{note}
The neural network must be first set to evaluation mode, as this influences the functionality of some neural network components such as the batch normalization layers.
:::

In [9]:
with torch.no_grad():
    trainer.model.eval()
    
    scaled_predicted_params = trainer.model(scaled_input)
    
print(scaled_predicted_params.shape)

torch.Size([64, 8])


Now we need to restore the scaled predicted parameters to their unscaled (physical) values. Since the predicted parameters are scaled with respect to the input prior bounds, these are also required for the rescaling. We can concatenate the `scaled_predicted_params` and `scaled_bounds` tensors along the last tensor axis and provide them as input to the `restore_params` method of the prior sampler object (which can be accessed as `trainer.loader.prior_sampler`), the output being an instance of the `ParametricParams` class.

In [10]:
restored_predictions = trainer.loader.prior_sampler.restore_params(torch.cat([scaled_predicted_params, scaled_bounds], dim=-1))
print(restored_predictions)

ParametricParams(batch_size=64, max_layer_num=2, device=cuda:0)


The physical predictions can then be accessed using the corresponding attribute for each parameter type.

In [11]:
pred_idx = 0

print(f'Predicted thicknesses: {restored_predictions.thicknesses[pred_idx]}')
print(f'Predicted roughnesses: {restored_predictions.roughnesses[pred_idx]}')
print(f'Predicted layers SLDs: {restored_predictions.slds[pred_idx]}')

Predicted thicknesses: tensor([185.6108, 392.6086], device='cuda:0', dtype=torch.float64)
Predicted roughnesses: tensor([26.7255, 16.5625, 18.7021], device='cuda:0', dtype=torch.float64)
Predicted layers SLDs: tensor([-18.5904,  -1.2606,  12.4019], device='cuda:0', dtype=torch.float64)


Based on the predictions, we can easily simulate the corresponding reflectivity curves by using the `reflectivity` method of the previously obtained `ParametricParams` object, which takes the q values as input:

In [12]:
predicted_curves = restored_predictions.reflectivity(q)

We can observe the input reflectivity curves alongside the curves corresponding to the neural network prediction. Additionally, we can print the prediction for each parameter alongside its ground truth value and its prior bounds.

In [13]:
from reflectorch import get_param_labels, get_density_profiles

@interact(i=(0, batch_size-1, 1))
def plot_refl_curve(i=0):
    fig, ax = plt.subplots(1,2,figsize=(12,6))

    ax[0].set_yscale('log')
    ax[0].set_ylim(0.5e-10, 5)

    ax[0].set_xlabel('q [$Å^{-1}$]', fontsize=20)
    ax[0].set_ylabel('R(q)', fontsize=20)

    ax[0].tick_params(axis='both', which='major', labelsize=15)
    ax[0].tick_params(axis='both', which='minor', labelsize=15)
    
    y_tick_locations = [10**(-2*i) for i in range(6)]
    ax[0].yaxis.set_major_locator(plt.FixedLocator(y_tick_locations))
        
    ax[0].plot(q[i].cpu().numpy(), predicted_curves[i].cpu().numpy() + 1e-10, c='r', lw=1, label='prediction')
    ax[0].scatter(q[i].cpu().numpy(), unscaled_noisy_curves[i].cpu().numpy() + 1e-10, c='b', s=2, label='input sim. curve')

    ax[0].legend(loc='upper right', fontsize=14)

    z_axis = torch.linspace(-200, 1000, 1000, device='cuda')
    _, sld_profile_gt, _ = get_density_profiles(
         simulated_data['params'].thicknesses, 
         simulated_data['params'].roughnesses,
         simulated_data['params'].slds,
         z_axis)
    
    _, sld_profile_pred, _ = get_density_profiles(
         restored_predictions.thicknesses, 
         restored_predictions.roughnesses,
         restored_predictions.slds,
         z_axis)
    
    ax[1].plot(z_axis.cpu().numpy(), sld_profile_pred[i].cpu().numpy(), c='r', label='prediction')
    ax[1].plot(z_axis.cpu().numpy(), sld_profile_gt[i].cpu().numpy(), c='b', label='ground truth')

    ax[1].set_xlabel('z [$Å$]', fontsize=20)
    ax[1].set_ylabel('SLD [$10^{-6} Å^{-2}$]', fontsize=20)
    ax[1].tick_params(axis='both', which='major', labelsize=15)
    ax[1].tick_params(axis='both', which='minor', labelsize=15)

    plt.tight_layout()


    for l, t, p, min_b, max_b in zip(
          get_param_labels(2),
          simulated_data['params'].parameters[i], 
          restored_predictions.parameters[i], 
          restored_predictions.min_bounds[i], 
          restored_predictions.max_bounds[i]
          ):
         print(f'{l.ljust(14)} --> True: {t:.2f} Predicted: {p:.2f}  Input prior bounds: ({min_b:.2f}, {max_b:.2f})')

interactive(children=(IntSlider(value=0, description='i', max=63), Output()), _dom_classes=('widget-interact',…

In [14]:
@interact(i=(0, batch_size-1, 1))
def plot_refl_curve(i=8):
    fig, ax = plt.subplots(1,2,figsize=(12,6))

    ax[0].set_yscale('log')
    ax[0].set_ylim(0.5e-10, 5)

    ax[0].set_xlabel('q [$Å^{-1}$]', fontsize=20)
    ax[0].set_ylabel('R(q)', fontsize=20)

    ax[0].tick_params(axis='both', which='major', labelsize=15)
    ax[0].tick_params(axis='both', which='minor', labelsize=15)
    
    y_tick_locations = [10**(-2*i) for i in range(6)]
    ax[0].yaxis.set_major_locator(plt.FixedLocator(y_tick_locations))
        
    ax[0].plot(q[i].cpu().numpy(), predicted_curves[i].cpu().numpy() + 1e-10, c='r', lw=1, label='prediction')
    ax[0].scatter(q[i].cpu().numpy(), unscaled_noisy_curves[i].cpu().numpy() + 1e-10, c='b', s=2, label='input sim. curve')

    ax[0].legend(loc='upper right', fontsize=14)

    z_axis = torch.linspace(-200, 1000, 1000, device='cuda')
    _, sld_profile_gt, _ = get_density_profiles(
         simulated_data['params'].thicknesses, 
         simulated_data['params'].roughnesses,
         simulated_data['params'].slds,
         z_axis)
    
    _, sld_profile_pred, _ = get_density_profiles(
         restored_predictions.thicknesses, 
         restored_predictions.roughnesses,
         restored_predictions.slds,
         z_axis)
    
    ax[1].plot(z_axis.cpu().numpy(), sld_profile_pred[i].cpu().numpy(), c='r', label='prediction')
    ax[1].plot(z_axis.cpu().numpy(), sld_profile_gt[i].cpu().numpy(), c='b', label='ground truth')

    ax[1].set_xlabel('z [$Å$]', fontsize=20)
    ax[1].set_ylabel('SLD [$10^{-6} Å^{-2}$]', fontsize=20)
    ax[1].tick_params(axis='both', which='major', labelsize=15)
    ax[1].tick_params(axis='both', which='minor', labelsize=15)

    plt.tight_layout()


    for l, t, p, min_b, max_b in zip(
          get_param_labels(2),
          simulated_data['params'].parameters[i], 
          restored_predictions.parameters[i], 
          restored_predictions.min_bounds[i], 
          restored_predictions.max_bounds[i]
          ):
         print(f'{l.ljust(14)} --> True: {t:.2f} Predicted: {p:.2f}  Input prior bounds: ({min_b:.2f}, {max_b:.2f})')

interactive(children=(IntSlider(value=8, description='i', max=63), Output()), _dom_classes=('widget-interact',…

In [15]:
@interact(i=(0, batch_size-1, 1))
def plot_refl_curve(i=13):
    fig, ax = plt.subplots(1,2,figsize=(12,6))

    ax[0].set_yscale('log')
    ax[0].set_ylim(0.5e-10, 5)

    ax[0].set_xlabel('q [$Å^{-1}$]', fontsize=20)
    ax[0].set_ylabel('R(q)', fontsize=20)

    ax[0].tick_params(axis='both', which='major', labelsize=15)
    ax[0].tick_params(axis='both', which='minor', labelsize=15)
    
    y_tick_locations = [10**(-2*i) for i in range(6)]
    ax[0].yaxis.set_major_locator(plt.FixedLocator(y_tick_locations))
        
    ax[0].plot(q[i].cpu().numpy(), predicted_curves[i].cpu().numpy() + 1e-10, c='r', lw=1, label='prediction')
    ax[0].scatter(q[i].cpu().numpy(), unscaled_noisy_curves[i].cpu().numpy() + 1e-10, c='b', s=2, label='input sim. curve')

    ax[0].legend(loc='upper right', fontsize=14)

    z_axis = torch.linspace(-200, 1000, 1000, device='cuda')
    _, sld_profile_gt, _ = get_density_profiles(
         simulated_data['params'].thicknesses, 
         simulated_data['params'].roughnesses,
         simulated_data['params'].slds,
         z_axis)
    
    _, sld_profile_pred, _ = get_density_profiles(
         restored_predictions.thicknesses, 
         restored_predictions.roughnesses,
         restored_predictions.slds,
         z_axis)
    
    ax[1].plot(z_axis.cpu().numpy(), sld_profile_pred[i].cpu().numpy(), c='r', label='prediction')
    ax[1].plot(z_axis.cpu().numpy(), sld_profile_gt[i].cpu().numpy(), c='b', label='ground truth')

    ax[1].set_xlabel('z [$Å$]', fontsize=20)
    ax[1].set_ylabel('SLD [$10^{-6} Å^{-2}$]', fontsize=20)
    ax[1].tick_params(axis='both', which='major', labelsize=15)
    ax[1].tick_params(axis='both', which='minor', labelsize=15)

    plt.tight_layout()


    for l, t, p, min_b, max_b in zip(
          get_param_labels(2),
          simulated_data['params'].parameters[i], 
          restored_predictions.parameters[i], 
          restored_predictions.min_bounds[i], 
          restored_predictions.max_bounds[i]
          ):
         print(f'{l.ljust(14)} --> True: {t:.2f} Predicted: {p:.2f}  Input prior bounds: ({min_b:.2f}, {max_b:.2f})')

interactive(children=(IntSlider(value=13, description='i', max=63), Output()), _dom_classes=('widget-interact'…

# Simplified use of a trained reflectorch model

In [16]:
import torch
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import interact
from reflectorch import EasyInferenceModel

torch.manual_seed(0); # set seed for reproducibility

The `EasyInferenceModel` class simplifies the inference step for single input reflectivity curves. We initialize an instance using the configuration file of a pretrained model.

In [17]:
inference_model = EasyInferenceModel(config_name='c1_trained')

Loading model c1_trained
Model c1_trained loaded. Number of parameters: 3.83 M
Model c1_trained is loaded.


We specify the prior bounds for the parameters as a list of tuples `(min_prior_bound, max_prior_bound)`:

In [18]:
prior_bounds = [(10., 500.), (10., 500.), #thicknesses (top to bottom)
                (0., 60.), (0., 60.), (0., 60.), #roughnesses (top to bottom)
                (-18., -16.), (-3., 0.), (9., 13.)] #slds (top to bottom)

The reflectivity curve and its corresponding q values should be Numpy array of Pytorch tensors:

In [19]:
reflectivity_curve = np.array([
        1.1851e+00, 1.3166e+00, 1.2792e+00, 1.1838e+00, 1.1931e+00, 3.9305e-01,
        2.3147e-01, 1.5170e-01, 1.0417e-01, 7.7329e-02, 7.1802e-02, 7.9045e-02,
        7.3709e-02, 9.2383e-02, 7.2035e-02, 8.3879e-02, 6.7362e-02, 3.8931e-02,
        3.2731e-02, 1.7164e-02, 6.0003e-03, 6.0304e-04, 4.4626e-04, 4.1207e-03,
        8.3723e-03, 1.5344e-02, 2.0189e-02, 2.3262e-02, 1.6822e-02, 2.0629e-02,
        1.6630e-02, 1.3288e-02, 8.9167e-03, 4.3387e-03, 2.2806e-03, 5.7447e-04,
        5.2052e-05, 3.4333e-04, 1.1657e-03, 1.3885e-03, 2.4396e-03, 2.6531e-03,
        2.6255e-03, 3.2867e-03, 2.2043e-03, 1.8492e-03, 1.2300e-03, 1.0895e-03,
        7.5618e-04, 5.9881e-04, 4.6538e-04, 3.8349e-04, 1.9684e-04, 1.1074e-04,
        7.1625e-05, 4.6080e-05, 8.6055e-05, 1.4261e-04, 2.6008e-04, 3.9065e-04,
        4.6900e-04, 4.9829e-04, 4.5922e-04, 4.7470e-04, 4.3097e-04, 3.1221e-04,
        2.2763e-04, 1.7086e-04, 5.9867e-05, 1.5550e-05, 1.8952e-07, 9.6534e-06,
        3.6830e-05, 6.7740e-05, 9.9643e-05, 1.2925e-04, 1.3112e-04, 1.2955e-04,
        1.1382e-04, 9.9504e-05, 7.7012e-05, 5.0792e-05, 3.9126e-05, 1.4859e-05,
        9.1510e-06, 3.9761e-06, 1.3064e-06, 2.2066e-06, 4.5884e-06, 8.6499e-06,
        1.3575e-05, 1.2752e-05, 1.9014e-05, 1.9985e-05, 2.0350e-05, 2.0645e-05,
        1.6298e-05, 1.2674e-05, 1.0688e-05, 8.1643e-06, 3.9810e-06, 2.2977e-06,
        5.9902e-07, 1.4382e-07, 4.5002e-07, 1.2234e-06, 2.3017e-06, 3.2775e-06,
        3.6404e-06, 4.4081e-06, 4.1741e-06, 3.3690e-06, 3.3674e-06, 2.4155e-06,
        1.9592e-06, 1.4078e-06, 9.6771e-07, 3.9546e-07, 1.8652e-07, 5.3041e-08,
        1.0580e-07, 2.0595e-07, 3.3874e-07, 5.7800e-07, 7.7420e-07, 5.6977e-07,
        6.6871e-07, 6.9172e-07
])

In [20]:
q_values = np.array([
        0.0201, 0.0211, 0.0222, 0.0232, 0.0242, 0.0252, 0.0262, 0.0273, 0.0283,
        0.0293, 0.0303, 0.0314, 0.0324, 0.0334, 0.0344, 0.0355, 0.0365, 0.0375,
        0.0385, 0.0396, 0.0406, 0.0416, 0.0426, 0.0436, 0.0447, 0.0457, 0.0467,
        0.0477, 0.0488, 0.0498, 0.0508, 0.0518, 0.0529, 0.0539, 0.0549, 0.0559,
        0.0570, 0.0580, 0.0590, 0.0600, 0.0611, 0.0621, 0.0631, 0.0641, 0.0651,
        0.0662, 0.0672, 0.0682, 0.0692, 0.0703, 0.0713, 0.0723, 0.0733, 0.0744,
        0.0754, 0.0764, 0.0774, 0.0785, 0.0795, 0.0805, 0.0815, 0.0825, 0.0836,
        0.0846, 0.0856, 0.0866, 0.0877, 0.0887, 0.0897, 0.0907, 0.0918, 0.0928,
        0.0938, 0.0948, 0.0959, 0.0969, 0.0979, 0.0989, 0.0999, 0.1010, 0.1020,
        0.1030, 0.1040, 0.1051, 0.1061, 0.1071, 0.1081, 0.1092, 0.1102, 0.1112,
        0.1122, 0.1133, 0.1143, 0.1153, 0.1163, 0.1174, 0.1184, 0.1194, 0.1204,
        0.1214, 0.1225, 0.1235, 0.1245, 0.1255, 0.1266, 0.1276, 0.1286, 0.1296,
        0.1307, 0.1317, 0.1327, 0.1337, 0.1348, 0.1358, 0.1368, 0.1378, 0.1388,
        0.1399, 0.1409, 0.1419, 0.1429, 0.1440, 0.1450, 0.1460, 0.1470, 0.1481,
        0.1491, 0.1501
])

We can invoke the `predict` method of the inference model to obtain the neural network predictions:

In [21]:
predicted_params = inference_model.predict(reflectivity_curve, q_values, prior_bounds)
print(predicted_params.squeeze().cpu().numpy())

[261.91821001 311.47147564  21.61200643  15.18360972  22.33419299
 -17.03783431  -1.59246207  11.33759486]


By invoking the method `predict_using_widget`, one can make use of an interactive Jupyter notebook widget for selecting the minimum and maximum prior bound for each parameter:

In [22]:
predicted_params = inference_model.predict_using_widget(reflectivity_curve, q_values)

Parameter ranges: {'thicknesses': [0.0, 500.0], 'roughnesses': [0.0, 60.0], 'slds': [-25.0, 25.0]}
Allowed widths of the prior bound intervals (max-min): {'thicknesses': [0.01, 500.0], 'roughnesses': [0.01, 60.0], 'slds': [0.01, 4.0]}
Please fill in the values of the minimum and maximum prior bound for each parameter and press the button!


Button(description='Make prediction', style=ButtonStyle())